# Image Classifier

## First we need to design the model


 #### We are going to use a 3 layer convolution and a 4 layer fully connected feed forward network to classify the images

In [1]:
import torch.nn as nn
import torch
from torch.nn import Module
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim import Adam
from torchvision.datasets import ImageFolder
import os 
import time
import copy
from torch.utils.data import random_split

class Flownet(Module):
    def __init__(self):
        super().__init__()

        self.conv_model = nn.Sequential(nn.Conv2d(3, 13, kernel_size=3, stride=1), #---> (13, 122, 122)
                              nn.ReLU(),

                              nn.Conv2d(13, 23, kernel_size=5, stride=1), #-----> (23, 118, 118)
                              nn.ReLU(),

                              nn.Conv2d(23, 33, kernel_size=5, stride=1), #-----> (33, 114, 114)
                              nn.ReLU()
                              )

        self.lin_model = nn.Sequential(nn.Linear(428868, 150),
                            nn.ReLU(),
                            nn.Linear(150, 80),
                            nn.ReLU(),
                            nn.Linear(80, 20),
                            nn.ReLU(),
                            nn.Linear(20, 4)
                            )

    def forward(self, x):
        x = self.conv_model(x)
        x = x.view(x.size()[0], -1)
        x = self.lin_model(x)
        x = torch.log_softmax(x, dim=1)
        return x


## Defining the dataloader and dataset

#### This cell we are transforming and normalizing the images and also defining a dataloader that will be responsible for feeding batches of images to the network.

In [2]:



train_transform = transforms.Compose([transforms.Resize((124,124)),
                               transforms.RandomHorizontalFlip(p=0.3),
                               transforms.RandomVerticalFlip(p=0.3),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),

                               
                               ])
test_transform = transforms.Compose([transforms.Resize((124, 124)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   ])
   


data_transforms = {'train': train_transform, 'test': test_transform}


data_dir = 'flocks/'


image_datasets = {x: ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Defining the train function

In [3]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            #if phase == 'train':
                #scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    torch.save(best_model_wts, 'models/v1.pth')
    return model

In [4]:
net = Flownet()
net = net.cuda()
# from torchvision.models import resnet34
from torch.nn import CrossEntropyLoss

# resnet = resnet34(pretrained=True)


#for param in resnet.parameters():
#    param.requires_grad = False

# resnet.fc = nn.Linear(512, 4)
# resnet = resnet.cuda()









optimizer = Adam(net.parameters(), lr=0.001)
#lr_sched = CosineAnnealingLR(optimizer, 20)
criterion = CrossEntropyLoss()



train_model(net, criterion, optimizer, num_epochs=15)

Epoch 0/14
----------
train Loss: 0.9310 Acc: 0.4752
test Loss: 0.7928 Acc: 0.5041

Epoch 1/14
----------
train Loss: 0.1739 Acc: 0.9142
test Loss: 0.0001 Acc: 1.0000

Epoch 2/14
----------
train Loss: 0.1258 Acc: 0.9732
test Loss: 0.0006 Acc: 1.0000

Epoch 3/14
----------
train Loss: 0.0241 Acc: 0.9972
test Loss: 0.0000 Acc: 1.0000

Epoch 4/14
----------


KeyboardInterrupt: 

In [15]:
net.eval()
image_dataset['train'].classes

NameError: name 'image_dataset' is not defined